In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_excel("bancodedados.xlsx")
df.head()

,Dates,BNTNB 6 05/15/2029 Govt,BNTNB 6 08/15/2050 Govt,ODF27 Comdty,ODF30 Comdty,USGG10YR Index,TY1 Comdty,CNH Curncy,USDBRL Curncy,EUR Curncy,SPX Index,IBOV Index
0,2023-01-02,NaN,3947.108,12.918,12.933,3.8748,112.296875,6.9270,5.3597,1.0667,3839.50,106376.02
1,2023-01-03,NaN,3947.108,13.155,13.134,3.7389,112.640625,6.9237,5.4627,1.0548,3824.14,104165.74
2,2023-01-04,NaN,3877.922,13.318,13.287,3.6827,113.187500,6.8995,5.4316,1.0604,3852.97,105334.46
3,2023-01-05,NaN,3877.922,13.055,13.025,3.7181,113.015625,6.8884,5.3529,1.0522,3808.10,107641.32
4,2023-01-06,NaN,3877.922,12.815,12.832,3.5580,114.218750,6.8303,5.2250,1.0644,3895.08,108963.70


In [3]:
rename_map = {
    "Dates": "Data",
    "BNTNB 6 05/15/2029 Govt": "NTNB-2029P",    # PU da NTN-B 2029
    "BNTNB 6 08/15/2050 Govt": "NTNB-2050P",    # PU da NTN-B 2050
    "ODF27 Comdty": "DI1F27_taxa",              # Taxa DI jan/2027 (% a.a.)
    "ODF30 Comdty": "DI1F30_taxa",              # Taxa DI jan/2030 (% a.a.)
    "USGG10YR Index": "T10_taxa",               # Yield to maturity Treasury 10y (% a.a.)
    "TY1 Comdty": "T10_fut",                    # Preço do contrato futuro de Treasury 10y
    "CNH Curncy": "CNY_USD",                    # Yuan por dólar (precisa multiplicar por USD/BRL para ter CNY em reais)
    "USDBRL Curncy": "USD_BRL",                 # Reais por dólar (já em BRL)
    "EUR Curncy": "EUR_USD",                    # Euros por dólar (precisa multiplicar por USD/BRL para ter EUR em reais)
    "SPX Index": "S&P500",                      # Pontos do índice S&P500 (precisa multiplicar por USD/BRL para ter em BRL)
    "IBOV Index": "IBOV"                        # Pontos do Ibovespa (já em BRL)
}
df.rename(columns=rename_map, inplace=True)
df.drop(df.index[-1], inplace=True)
df["Data"] = pd.to_datetime(df["Data"])

In [4]:
# Definir vencimentos
venc_DI1F27 = datetime(2027, 1, 1)
venc_DI1F30 = datetime(2030, 1, 1)
prazo_T10 = 10  # anos

# Função para dias úteis (simplificação: considerar todos dias como úteis para exemplo)
def dias_uteis(data_inicial, data_final):
    return np.busday_count(data_inicial.date(), data_final.date())

# Converter DI1F27 e DI1F30 de taxa (% a.a.) para preço usando fórmula PU = 1000 / (1 + r)^(d/252)
for col, venc in [("DI1F27_taxa", venc_DI1F27), ("DI1F30_taxa", venc_DI1F30)]:
    dias_restantes = df["Data"].apply(lambda d: dias_uteis(d, venc))
    df[col.replace("_taxa", "_preco")] = 1000 / (1 + df[col] / 100) ** (dias_restantes / 252)

# Converter T10_taxa (Treasury 10y) como se fosse zero cupom: Preço ≈ 100 / (1 + r)^10
df["T10_preco"] = 100 / (1 + df["T10_taxa"] / 100) ** prazo_T10

# Remover colunas de taxa
df.drop(columns=["DI1F27_taxa", "DI1F30_taxa", "T10_taxa"], inplace=True)

In [5]:
df.head()

,Data,NTNB-2029P,NTNB-2050P,T10_fut,CNY_USD,USD_BRL,EUR_USD,S&P500,IBOV,DI1F27_preco,DI1F30_preco,T10_preco
0,2023-01-02,NaN,3947.108,112.296875,6.9270,5.3597,1.0667,3839.50,106376.02,604.518636,414.246011,68.375103
1,2023-01-03,NaN,3947.108,112.640625,6.9237,5.4627,1.0548,3824.14,104165.74,599.584389,409.142938,69.276129
2,2023-01-04,NaN,3877.922,113.187500,6.8995,5.4316,1.0604,3852.97,105334.46,596.318598,405.358653,69.652550
3,2023-01-05,NaN,3877.922,113.015625,6.8884,5.3529,1.0522,3808.10,107641.32,602.368815,412.409633,69.415184
4,2023-01-06,NaN,3877.922,114.218750,6.8303,5.2250,1.0644,3895.08,108963.70,607.970981,417.740178,70.495835


In [6]:
df = df.iloc[276:].reset_index(drop=True) #276 temos NTNB-2029P com NaN
df.head()

,Data,NTNB-2029P,NTNB-2050P,T10_fut,CNY_USD,USD_BRL,EUR_USD,S&P500,IBOV,DI1F27_preco,DI1F30_preco,T10_preco
0,2024-01-23,4341.021,4491.526,111.218750,7.1674,4.9503,1.0854,4864.60,128262.52,748.928889,540.570665,66.729272
1,2024-01-24,4341.021,4499.201,110.968750,7.1623,4.9328,1.0885,4868.55,127815.70,749.625818,541.326949,66.422449
2,2024-01-25,4352.082,4506.861,111.312500,7.1799,4.9166,1.0846,4894.16,128168.73,751.943274,543.683846,66.792107
3,2024-01-26,4352.082,4520.395,111.031250,7.1885,4.9109,1.0853,4890.97,128967.32,753.201208,545.169552,66.670984
4,2024-01-29,4352.082,4512.293,111.515625,7.1875,4.9480,1.0833,4927.93,128502.66,752.378592,543.387151,67.077603
